<a href="https://colab.research.google.com/github/valkryhx/Firefly/blob/master/gptq_lora_baichuan7b_vicuna_v2_%E6%94%AF%E6%8C%81%E5%88%86%E5%B8%83%E5%BC%8F%E8%AE%AD%E7%BB%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/valkryhx/Firefly

Cloning into 'Firefly'...
remote: Enumerating objects: 610, done.
remote: Counting objects: 100% (442/442), done.
remote: Compressing objects: 100% (243/243), done.
remote: Total 610 (delta 248), reused 308 (delta 199), pack-reused 168
Receiving objects: 100% (610/610), 6.37 MiB | 14.16 MiB/s, done.
Resolving deltas: 100% (336/336), done.


In [ ]:
%cd Firefly/
!pip install -r requirements.txt

/content/Firefly
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-f_k5on_1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-f_k5on_1
  Resolved https://github.com/huggingface/peft.git to commit e06d94ddeb6c70913593740618df76908b918d66
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
pwd

'/content/Firefly'

In [ ]:
%cd Firefly/

[Errno 2] No such file or directory: 'Firefly/'
/content/Firefly


# gptq + lora 训练： 从base model开始

In [ ]:
!git pull --all --force
! python train_gptq_lora.py  --train_args_file train_args/qlora/baichuan-7b-sft-qlora.json \
              --output_dir  output/baichuan-gptq-v1

Fetching origin
Already up to date.
[2023-07-20 14:24:15,988] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-07-20 14:24:21.700005: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-20 14:24:26.412 | INFO     | __main__:main:347 - myargs=QLoRAArguments(max_seq_length=512, train_file='./data/dummy_data_2.jsonl', model_name_or_path='fireballoon/baichuan-vicuna-chinese-7b-gptq', task_type='', eval_file='', lora_rank=8, lora_alpha=32, lora_dropout=0.05)
2023-07-20 14:24:26.412 | INFO     | __main__:init_components:137 - Initializing components...
2023-07-20 14:24:26.412 | INFO     | __main__:init_components:227 - GPTQ 模型加载开始: fireballoon/baichuan-vicuna-chinese-7b-gptq
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` 

# gptq + lora 继续训练：从base model挂载上次的lora adapter 继续lora训练

#  代码中居然是str类型？不能写bool 否则只要传参就是True 真是玄学
# parser.add_argument("--ddp_find_unused_parameters", type=str, default=False)
# 传参 --ddp_find_unused_parameters False

In [ ]:
!git pull --all --force

!TORCH_DISTRIBUTED_DEBUG=DETAIL torchrun --nproc-per-node=1 train_gptq_lora.py  --train_args_file train_args/qlora/baichuan-7b-sft-qlora.json \
              --peft_path  output/baichuan-gptq-v1/final   \
              --output_dir  output/baichuan-gptq-v2 \
              --ddp_find_unused_parameters False

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 711 bytes | 711.00 KiB/s, done.
From https://github.com/valkryhx/Firefly
   6e8dd40..c48cd8d  master     -> origin/master
Updating 6e8dd40..c48cd8d
Fast-forward
 train_gptq_lora.py | 6 +++---
 1 file changed, 3 insertions(+), 3 deletions(-)
[2023-07-20 16:59:51,679] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-07-20 16:59:57.305617: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-20 17:00:02.714 | INFO     | __main__:setup_everything:115 - args.ddp_find_unused_parameters=False
2023-07-20 17:00:02.714 | INFO     | __main__:setup_everything:116 - mark 0   ddp_find_unused_parameters = False
2023-07-20 17:00:02.721 | INFO     | __main__:setup_everyt

In [ ]:
F